# Resnet example using gRPC Client

This example demonstrates using our gRPC client to perform inference with the pretrained ResNet18 model.

Clone the repo with notebooks and corresponding data. 

In [ ]:
!git clone https://github.com/TurboML-Inc/colab-notebooks.git

Set up the environment and install TurboML's SDK. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!bash colab-notebooks/install_turboml.sh

The kernel should now be restarted with TurboML's SDK installed.

In [ ]:
cd colab-notebooks

Login to your TurboML instance.

In [ ]:
import pandas as pd
import turboml as tb
tb.init(backend_url=BACKEND_URL, api_key=API_KEY)

In [ ]:
!pip install kagglehub

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from resnet_grpc_server import serve

### Start gRPC server for pretrained Resnet18 from jupyter-notebook

In [ ]:
import threading


def run_server_in_background(url):
    serve(url)  # This will start the gRPC server


# Start the server in a separate thread
url = "0.0.0.0:50021"
server_thread = threading.Thread(
    target=run_server_in_background, args=(url,), daemon=True
)
server_thread.start()

print("gRPC server is running in the background...")

### Load image Dataset from Kaggle

In [ ]:
import kagglehub
import shutil

# Download latest version
target_path = "./data/animal-image-classification-dataset"
path = kagglehub.dataset_download("borhanitrash/animal-image-classification-dataset")
shutil.move(path, target_path)

print("Dataset stored in:", target_path)

In [ ]:
animal_dataset = datasets.ImageFolder(root=target_path, transform=transforms.ToTensor())
data_loader = DataLoader(animal_dataset, batch_size=32, shuffle=True)
images, labels = next(iter(data_loader))

### Convert images into bytes array.

In [ ]:
import io

images_test = []
labels_test = []

for image_tensor, label in zip(images, labels, strict=False):
    image = transforms.ToPILImage()(image_tensor)
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format="JPEG")
    binary_image = img_byte_arr.getvalue()

    images_test.append(binary_image)
    labels_test.append(label.item())

image_dict_test = {"images": images_test}
label_dict_test = {"labels": labels_test}
image_df_test = pd.DataFrame(image_dict_test)
label_df_test = pd.DataFrame(label_dict_test)
image_df_test.reset_index(inplace=True)
label_df_test.reset_index(inplace=True)

print(f"Processed {len(images_test)} images.")
print(f"Image DataFrame shape: {image_df_test.shape}")
print(f"Label DataFrame shape: {label_df_test.shape}")

In [ ]:
image_df_test = image_df_test.reset_index(drop=True)
label_df_test = label_df_test.reset_index(drop=True)

In [ ]:
images_test = tb.PandasDataset(
    dataframe=image_df_test, key_field="index", streaming=False
)
labels_test = tb.PandasDataset(
    dataframe=label_df_test, key_field="index", streaming=False
)

In [ ]:
imaginal_fields = ["images"]
features_test = images_test.get_input_fields(imaginal_fields=imaginal_fields)
targets_test = labels_test.get_label_field(label_field="labels")

### Using TurboML Client to request gRPC server

In [ ]:
grpc_model = tb.GRPCClient(
    server_url="0.0.0.0:50021",
    connection_timeout=10000,
    max_request_time=10000,
    max_retries=1,
)

In [ ]:
model_trained = grpc_model.learn(features_test, targets_test)

In [ ]:
outputs = model_trained.predict(features_test)

In [ ]:
outputs  # {class,probability}